In [1]:
import pandas as pd
import sys  
sys.path.insert(1, '/home/tb24/projects/llm-data-aug')

# Path
import os

# Get the project root directory (one level up from the notebook directory)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Define the data path
data_path = os.path.join(project_root, "data")

### Concat checkpoints to form a complete dataset

In [10]:
dir = "llm_generated/gemini-2.0-flash"

files = os.listdir(os.path.join(data_path, dir))
# Negative sentiment
# files = [f for f in files if f.endswith(".csv") and "auggpt_augmented_user_reviews_negative" in f]

# Neutral sentiment
files = [f for f in files if f.endswith(".csv") and "auggpt_augmented_user_reviews_neutral" in f]   

df = pd.concat([pd.read_csv(os.path.join(data_path, dir, f)) for f in files])
df.head()



,Review,Sentiment
0,không được mát lắm,2
1,Nó không thực sự mát.,2
2,Nó không mát như tôi mong đợi.,2
3,Độ mát của nó không được tốt lắm.,2
4,Tôi không thấy nó mát lắm.,2


In [11]:
# df.to_csv(os.path.join(data_path, dir, "auggpt_augmented_user_reviews_negative.csv"), index=False)
df.to_csv(os.path.join(data_path, dir, "auggpt_augmented_user_reviews_neutral.csv"), index=False)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1806 entries, 0 to 119
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     1806 non-null   object
 1   Sentiment  1806 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 42.3+ KB


### Examine and clean the Generated Data (with AugGPT)


In [2]:
df = pd.read_csv(os.path.join(data_path, "llm_generated/gemini-2.0-flash", "auggpt_upsampled_user_reviews.csv"))
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/tb24/projects/llm-data-aug/data/llm_generated/gemini-2.0-flash/auggpt_upsampled_user_reviews.csv'

In [12]:
# The number of records for each sentiment
label_mapping = {'Positive': 1, 'Neutral': 2, 'Negative': 0}
df["Sentiment"].value_counts()


Sentiment
 1    2106
 0     580
 2     337
-1      18
 3       3
 5       2
 4       2
Name: count, dtype: int64

In [13]:
# Remove invalid sentiment: -1, 3, 4, 5
df = df[df["Sentiment"].isin([0, 1, 2])]
df["Sentiment"].value_counts()

Sentiment
1    2106
0     580
2     337
Name: count, dtype: int64

In [16]:
# Remove the Majority class, in this case it is Positive - 1
df = df[df["Sentiment"] != 1]
df["Sentiment"].value_counts()
df.reset_index(drop=True, inplace=True)
df

,Review,Sentiment
0,Thật kinh khủng khi tôi phát hiện tóc trong th...,0
1,Tôi rất bực mình khi tìm thấy tóc trong đồ ăn ...,0
2,McDonald's đang cho khách hàng ăn phải tóc ư? ...,0
3,Tôi không thể tin được McDonald's lại để tóc r...,0
4,"Tai nạn đáng tiếc xảy ra tại McDonald's, tôi t...",0
...,...,...
912,Great value for the price. This product delive...,2
913,I love the portability of this device. It's pe...,2
914,"ok sạch sẽ, nhiệt tình, chu đáo",2
915,I order from them.,2
